In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Load Oxford Pets dataset from TFDS
(train_ds, test_ds), info = tfds.load('oxford_iiit_pet:3.*.*', split=['train', 'test'], with_info=True)

# Define preprocessing functions
def preprocess_image(data):
    image = tf.image.resize(data['image'], (32, 32))  # Resize image
    image = tf.cast(image, tf.float32) / 255.0  # Normalize pixel values to [0, 1]
    label = tf.image.resize(data['segmentation_mask'], (32, 32))  # Resize label
    label = tf.cast(label, tf.int32)  # Ensure labels are integers
    
    # Convert labels to one-hot categorical
    num_classes = info.features['label'].num_classes
    label = tf.one_hot(label, depth=num_classes)
    
    # Reshape labels
    label = tf.reshape(label, (32, 32, num_classes))  # Reshape to (32, 32, num_classes)
    
    return image, label

# Apply preprocessing to datasets
train_ds = train_ds.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_ds = test_ds.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)


In [2]:
for image, label in train_ds.take(1):
    print("Image shape:", image.shape)
    print("Label shape:", label.shape)

Image shape: (32, 32, 3)
Label shape: (32, 32, 37)


In [13]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model

def create_segmentation_model(input_shape, num_classes):
    # Load ResNet50 backbone
    resnet50_base = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    # Add segmentation-specific layers on top of ResNet50
    x = resnet50_base.output
    x = tf.keras.layers.UpSampling2D(size=(32, 32))(x)  # Upsample to match original image size
    x = tf.keras.layers.Conv2D(num_classes, kernel_size=(1, 1), activation='softmax')(x)

    # Create segmentation model
    model = tf.keras.models.Model(inputs=resnet50_base.input, outputs=x)
    return model

# Instantiate the model
input_shape = (32, 32, 3)  # Adjust input shape as per your dataset
num_classes = info.features['label'].num_classes
model = create_segmentation_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=10, monitor='loss', verbose=1)
model.fit(train_ds.batch(32), epochs=100,callbacks=[early_stopping_monitor])

Epoch 1/100
115/115 [==============================] - 15s 93ms/step - loss: 1.2946 - accuracy: 0.5301
Epoch 2/100
115/115 [==============================] - 11s 94ms/step - loss: 0.9293 - accuracy: 0.5855
Epoch 3/100
115/115 [==============================] - 11s 95ms/step - loss: 0.8935 - accuracy: 0.6028
Epoch 4/100
115/115 [==============================] - 11s 95ms/step - loss: 0.8849 - accuracy: 0.6097
Epoch 5/100
115/115 [==============================] - 11s 93ms/step - loss: 0.8801 - accuracy: 0.6155
Epoch 6/100
115/115 [==============================] - 10s 87ms/step - loss: 0.8822 - accuracy: 0.6090
Epoch 7/100
115/115 [==============================] - 11s 95ms/step - loss: 0.8759 - accuracy: 0.6115
Epoch 8/100
115/115 [==============================] - 11s 98ms/step - loss: 0.8650 - accuracy: 0.6210
Epoch 9/100
115/115 [==============================] - 11s 98ms/step - loss: 0.8500 - accuracy: 0.6291
Epoch 10/100
115/115 [==============================] - 11s 97ms/step - l

In [15]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(test_ds.batch(32))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

115/115 [==============================] - 9s 67ms/step - loss: 0.9845 - accuracy: 0.5411
Test Loss: 0.9845007658004761
Test Accuracy: 0.5410877466201782


In [16]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model
import keras

def create_segmentation_model(input_shape, num_classes):
    # Load ResNet50 backbone
    resnet50_base = keras.models.load_model(r"F:\Pre-Trained_Models\Pre-Trained_Models\oxf_32_model.h5")
    
    # Add segmentation-specific layers on top of ResNet50
    x = resnet50_base.layers[-4].output
    x = tf.keras.layers.UpSampling2D(size=(32, 32))(x)  # Upsample to match original image size
    x = tf.keras.layers.Conv2D(num_classes, kernel_size=(1, 1), activation='softmax')(x)

    # Create segmentation model
    model = tf.keras.models.Model(inputs=resnet50_base.input, outputs=x)
    return model

# Instantiate the model
input_shape = (32, 32, 3)  # Adjust input shape as per your dataset
num_classes = info.features['label'].num_classes
model = create_segmentation_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping_monitor = EarlyStopping(patience=10, monitor='loss', verbose=1)
model.fit(train_ds.batch(32), epochs=100,callbacks=[early_stopping_monitor])

Epoch 1/100
115/115 [==============================] - 15s 92ms/step - loss: 1.0559 - accuracy: 0.5731
Epoch 2/100
 98/115 [========================>.....] - ETA: 1s - loss: 0.9041 - accuracy: 0.6021

In [ ]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(test_ds.batch(32))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model
import keras

def create_segmentation_model(input_shape, num_classes):
    # Load ResNet50 backbone
    resnet50_base = keras.models.load_model(r"F:\Pre-Trained_Models\Pre-Trained_Models\rnd_32_model.h5")
    
    # Add segmentation-specific layers on top of ResNet50
    x = resnet50_base.layers[-4].output
    x = tf.keras.layers.UpSampling2D(size=(32, 32))(x)  # Upsample to match original image size
    x = tf.keras.layers.Conv2D(num_classes, kernel_size=(1, 1), activation='softmax')(x)

    # Create segmentation model
    model = tf.keras.models.Model(inputs=resnet50_base.input, outputs=x)
    return model

# Instantiate the model
input_shape = (32, 32, 3)  # Adjust input shape as per your dataset
num_classes = info.features['label'].num_classes
model = create_segmentation_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=10, monitor='loss', verbose=1)
model.fit(train_ds.batch(32), epochs=100,callbacks=[early_stopping_monitor])

In [ ]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(test_ds.batch(32))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)